In [1]:
!pip install pandas numpy matplotlib seaborn scikit-learn nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 475.3 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import zipfile

# Unzip and load the dataset
with zipfile.ZipFile("complaints.csv.zip", 'r') as zip_ref:
    zip_ref.extractall()

# Replace this with the actual CSV file name inside the ZIP
df = pd.read_csv("complaints.csv")
df.head()


/var/folders/c8/v746sghs5yl5t82hb93fz36h0000gn/T/ipykernel_38115/2374209070.py:9: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("complaints.csv")


,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2025-06-09,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",NC,27613,NaN,NaN,Web,2025-06-09,In progress,Yes,NaN,13964976
1,2025-06-09,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",TX,77306,NaN,NaN,Web,2025-06-09,In progress,Yes,NaN,13964977
2,2025-06-09,Credit reporting or other personal consumer re...,Credit reporting,Problem with a company's investigation into an...,Their investigation did not fix an error on yo...,NaN,NaN,"EQUIFAX, INC.",FL,34116,NaN,NaN,Web,2025-06-09,In progress,Yes,NaN,13964981
3,2025-06-09,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",FL,33897,NaN,NaN,Web,2025-06-09,In progress,Yes,NaN,13964982
4,2025-06-09,Credit reporting or other personal consumer re...,Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,NaN,"EQUIFAX, INC.",FL,33315,NaN,NaN,Web,2025-06-09,In progress,Yes,NaN,13964983


In [3]:
#Filter Relevant Columns and Categories
df = df[['Product', 'Consumer complaint narrative']].dropna()

# Target categories
target_categories = [
    "Credit reporting, repair, or other",
    "Debt collection",
    "Consumer Loan",
    "Mortgage"
]

# Filter only required categories
df = df[df['Product'].isin(target_categories)]

# Map to labels
label_map = {cat: idx for idx, cat in enumerate(target_categories)}
df['label'] = df['Product'].map(label_map)

df.head()


,Product,Consumer complaint narrative,label
197,Mortgage,I purchased a home from Illinois to move to Lo...,3
237,Debt collection,I've complained about procollect and this fals...,1
240,Debt collection,Credit Collections obtained access to my consu...,1
1844,Debt collection,"First of all, I never received any email or of...",1
3124,Debt collection,I was not served to appear in court for either...,1


In [4]:
# Text Preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', str(text))
    text = text.lower()
    words = [stemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(words)

df['cleaned_text'] = df['Consumer complaint narrative'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rishwanth/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Split and Vectorize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(max_features=3000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [ ]:
# Model Training and Comparison
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier()
}

for name, model in models.items():
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    print(f"\n--- {name} ---")
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print(classification_report(y_test, y_pred))



--- Logistic Regression ---
Accuracy: 0.9668370051525625
              precision    recall  f1-score   support

           1       0.97      0.98      0.98     66659
           2       0.76      0.50      0.61      1875
           3       0.96      0.96      0.96     25788

    accuracy                           0.97     94322
   macro avg       0.90      0.81      0.85     94322
weighted avg       0.97      0.97      0.97     94322


--- Naive Bayes ---
Accuracy: 0.9503085176310935
              precision    recall  f1-score   support

           1       0.97      0.97      0.97     66659
           2       0.72      0.27      0.40      1875
           3       0.91      0.95      0.93     25788

    accuracy                           0.95     94322
   macro avg       0.86      0.73      0.76     94322
weighted avg       0.95      0.95      0.95     94322



In [ ]:
# Predict Function
def predict_category(text, model=models['Logistic Regression']):
    cleaned = clean_text(text)
    vector = vectorizer.transform([cleaned])
    pred = model.predict(vector)[0]
    return [k for k, v in label_map.items() if v == pred][0]

# Example prediction
predict_category("I have a problem with my credit report.")
